# <p style="background-color:#0b032d;font-family:newtimeroman;color:#f4c095;font-size:150%;text-align:center ;border-radius:10px 10px;"> Análisis de Clústeres con K-Means </p>

## Importar librerias

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import copy

## Importar data

In [ ]:
# Importar datos
# r : evitar problemas con la barras invertidas
data = pd.read_csv(r'data\telefonica.csv', sep= ",", index_col="IDCliente")
data.head(3)

In [ ]:
data.describe()

Nota: <br>

* Aparentemente no hay datos faltantes en ninguna de las variables
* En el data set contiene en igual proporcion tanto el genero masculino como femenino
* En promedio los clientes poseen dos años y 6 meses, no son clientes nuevos
* El %90,3 de los clientes tienen linea telefonica 
* El %42,1 de los clientes cuentan con multiples servicios 
* El %26,5 de los clientes abandonan la empresa.

## Mapa de calor

In [ ]:
plt.figure(figsize= (20, 10))
sns.heatmap(
    data= data.corr(),
    annot= True,
    cmap="RdBu",
    vmin= -1,
    vmax= 1
)
plt.show()

Nota:

*Baja probabilidad de no abandono*<br>

* **tipo contrato:** Los clientes con contrato a un año o 2 años tienen menos probabilidad de abandono
* **meses:**: Los clientes antiguos tienen menos probabilidades de abandono
* **casados:**
* **dependientes** o personas a su cargo
* **Soporte:** Clientes con soporte tecnico 

*Mayor probabilidad de abandono*<br>

* **Personas con internet** 
* **Tercera edad**
* **factura electronica** 
* **MetodoPago:** Entre más complejo sea el metodo de pago mas probabilidades de abandono
* **Factura electronica:** 
* **Striming tv** y **Deportes** : cada una pesa alrededor del %6 respectivamente 

## Estandarizar datos

In [225]:
# crear objeto standar
standar = StandardScaler()

# Estandarizar los datos
data_std = standar.fit_transform(data)

In [ ]:
# Mostrar datos, Estos datos muestras a cuantas desviaciones estandar están los datos 
data_std

## Determinar  K con el método del codo

Within cluster sum of squares = suma de cuadrados dentro delos clusteres (varianza)

In [227]:
# calcular el within cluster sum of square
wcss = []
for i in range(1, 11):
    kmean = KMeans(n_clusters= i, init= "k-means++", random_state= 99)
    kmean.fit(data_std)
    wcss.append(kmean.inertia_)

In [ ]:
# Grafica del codo
plt.figure( figsize = (10,5))
plt.plot(range(1,11), wcss, marker = "o")
plt.xlabel('Núero de cluster')
plt.ylabel('Suma de las distancias al cuadrado dentro del clusteres')
plt.show()

**Nota:**
+ Apartir de cluster "4" la variaza no cambia en gran medida.
+ Los candidatos sestá en el cluster 4 y 5, eneste caso se tomará el 4

# Implementando K-Means Cluster 

In [ ]:
# Crear el objeto cluster
cluster_4 = KMeans(n_clusters=4, init="k-means++", random_state=99)

# Hacer el fit con los datos standarizados
cluster_4.fit(data_std)

## Resultados<br>
- copiar los daros originales para gregar etiquetas
- esto es para optener las etiquetas y saber cual cluster le corresponde a cada uno

In [230]:
# Copiar datos para incertar el laber "etiqueta cluster"
data_copy = copy.deepcopy(data)

In [231]:
# Agregar etiquetas al dataframe original
data_copy['Cluster']= cluster_4.labels_

In [ ]:
# Mostrar datos
data_copy.head(3)

In [ ]:
# Analizar la media de los datos teniendo en cunata el numero de cluster
# Esto se hace sacando la media de los datos y agrupando por cluster

df_analisis = data_copy.groupby('Cluster').mean()
# Mostrar la media de los grupos/Clusteres
df_analisis

In [234]:
# Crear un archivo excel para analizar con mayor facilidad los resultados y guardar en carpeta data
# r : evitar problemas con la barras invertidas
# df_analisis.to_excel(r"data/resumen cluster.xlsx")

**Tip para evaluar resultados:** <br>

1. Identificar valores promedios más altos y bajos de las variables relevates compandos con los demas closteres.
2. Recapitular o hacer un resumen donde de los resultados
3. Con base a lo anteriror colocar label o etiquetas a los clusteres Ejemplo: Adultos mayores, madres con ingresos altos etc.
---



**Análisis de resultados:** <br>

Generalidades:
- El género no es relevante


**Cluster 0:** <br>
1. Es el grupo con mayor proporción de personas mayores.
2. No tienen interés en servicios telefónicos.
3. No les interesan las líneas múltiples; solo quieren internet básico.
4. Son el segundo grupo que contrata más servicios adicionales.
5. Es el que tiene una menos probabilidad de abandono<br>

**"Adultos mayores con Internet Básico"** <br>
Descripción: Este clúster está compuesto por personas mayores que no tienen interés en servicios telefónicos o líneas múltiples, y solo buscan un servicio de internet básico. Contratan una cantidad moderada de servicios adicionales. <br><br>

**Cluster 1:** <br>

1.Es el segundo grupo con mayor proporción de personas casadas.
2. Tienen la mayor cantidad de dependientes.
3. Son el segundo grupo más antiguo.
4. Tienen una alta cantidad de líneas múltiples.
5. No les interesa mucho los servicios adicionales como TV, películas, seguridad, respaldo, entre otros; solo priorizan el internet de alta velocidad.
6. Son el segmento con el contrato más largo.
7. Prefieren facturas en papel.
8. Prefieren métodos de pago físicos sobre los electrónicos.
9. Generan menos ingresos para la empresa. <br>

**"Familias Tradicionales con Alta Estabilidad"**
Descripción: Este clúster está formado por personas casadas con muchos dependientes y una preferencia por el internet de alta velocidad. Tienen contratos largos y prefieren métodos de pago y facturas físicos. Generan menos ingresos para la empresa. <br><br>

**Cluster 2:** <br>

1. Es el grupo más joven.
2. En su mayoría son solteros.
3. Tienen pocos dependientes.
4. Son el grupo con menos tiempo en la empresa.
5. Valoran mucho tener fibra óptica.
6. Son el segundo grupo con más líneas múltiples.
7. Tienen el contrato más corto.
8. Prefieren facturas electrónicas.
9. Son el segundo grupo que más ingresos genera para la empresa.
10. El cliente que la mayor probabilidad de abandona <br>

**"Jóvenes Solteros / Gamers**
Descripción: Este clúster consiste en personas jóvenes y solteras con pocos dependientes, que valoran mucho tener fibra óptica y prefieren facturas electrónicas. Son el segundo grupo en cuanto a líneas múltiples y generan buenos ingresos para la empresa <br><br>


**Cluster 3:** <br>

1. La mayoría están casados.
2. Tienen una alta cantidad de dependientes.
3. Son el grupo más antiguo.
4. Tienen el mayor número de líneas múltiples.
5. Valoran tener buena calidad de internet.
6. Les importa mucho tener soporte técnico, seguridad, TV y películas.
7. Tienen los contratos más largos.
8. Prefieren utilizar medios electrónicos para la notificación de su recibo.
9. Prefieren pagar por medios electrónicos como transferencias bancarias o tarjetas de crédito.
10. Son el grupo que más ingresos genera para la empresa. <br>

**"Usuarios Rentables"**
Descripción: Este clúster está compuesto por personas casadas con muchos dependientes, que valoran tener buena calidad de internet, soporte técnico, seguridad, TV y películas. Tienen contratos largos y prefieren medios electrónicos tanto para la facturación como para los pagos. Generan los mayores ingresos para la empresa.